In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

%matplotlib inline

In [2]:
df = pd.read_csv('joined_labels.csv')

In [3]:
nb_labels = 3

In [6]:
df.head()

,reviewText,labels,product_category,num_labels
0,Used it 3 times...now leaking oil...what a POS.,Quality,mower,1
1,I don't like how the throttle works. It is har...,Design/Functionality,mower,2
2,Here's the real deal on these new cheap Poulan...,Design/Functionality,mower,2
3,If you are looking for a manual push mower you...,Design/Functionality,mower,2
4,DO NOT BUY\nI used this twice and it stopped w...,Quality,mower,1


In [20]:
df = df[df['num_labels']<= (nb_labels -1)]

In [21]:
df.head()

,reviewText,labels,product_category,num_labels,cln_review
0,Used it 3 times...now leaking oil...what a POS.,Quality,mower,1,used 3 timesnow leaking oilwhat pos
1,I don't like how the throttle works. It is har...,Design/Functionality,mower,2,dont like throttle works hard hand constantly ...
2,Here's the real deal on these new cheap Poulan...,Design/Functionality,mower,2,heres real deal new cheap poulan mowers bought...
3,If you are looking for a manual push mower you...,Design/Functionality,mower,2,looking manual push mower want stay away one w...
4,DO NOT BUY\nI used this twice and it stopped w...,Quality,mower,1,buyi used twice stopped working entirely stopp...


In [22]:
import nltk

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [25]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [26]:
df['cln_review'] = df['reviewText'].apply(clean_text)

/tmp/ipykernel_9629/64232819.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cln_review'] = df['reviewText'].apply(clean_text)


In [27]:
df.index.values

array([   0,    1,    2, ..., 2598, 2599, 2600])

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.cln_review.values, 
                                                  df.num_labels.values, 
                                                  test_size=0.10, 
                                                  random_state=42, 
                                                  stratify=df.num_labels.values)

X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size=0.111, 
                                                  random_state=42, 
                                                  stratify=y_train)

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [30]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_val)

my_tags = ['0', '1', '2']
print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred,target_names=my_tags))

accuracy 0.8504273504273504
              precision    recall  f1-score   support

           0       0.93      0.89      0.91        73
           1       0.77      0.88      0.82        85
           2       0.89      0.78      0.83        76

    accuracy                           0.85       234
   macro avg       0.86      0.85      0.85       234
weighted avg       0.86      0.85      0.85       234

CPU times: user 9.79 ms, sys: 0 ns, total: 9.79 ms
Wall time: 9.25 ms


In [32]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

my_tags = ['0', '1', '2']
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.8333333333333334
              precision    recall  f1-score   support

           0       0.90      0.89      0.90        73
           1       0.77      0.89      0.83        85
           2       0.86      0.71      0.78        76

    accuracy                           0.83       234
   macro avg       0.84      0.83      0.83       234
weighted avg       0.84      0.83      0.83       234

CPU times: user 10.7 ms, sys: 12 µs, total: 10.7 ms
Wall time: 10 ms
